In [1]:
import pyqubo as pq
import numpy as np
import qubovert as qv

In [3]:
def chi(precision, number):
    # number must be between 0 and 2
    q = np.zeros((number.size, precision))
    print(q)
    for i in range(len(number)):
        for r in range(precision):
            if number[i] < 2**(-r):
                q[i,r]=0
            else:
                q[i,r]=1
                number[i] -= 2**(-r)
    return q

test = np.array([.126,.6721,1.46])
chi(5, test)




[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


array([[0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 0.],
       [1., 0., 1., 1., 1.]])

In [60]:


def H(M, y, R = 2, a = -1, b = 3):
    '''
    M and y come from problem Mx-y
    R is precision, ie number of qubits
    a, b endpoints of desired range, a < b, [a,b)
    '''
    
    # Think I need Q to be upper triangular, how best to do this?
    # Need to add in the multiplication by pho part as well!!
    
    # Save the dimension of problem as N
    N = np.shape(y)[0]

    # Define the midpoint of the range, hard coded atm
    gamma = (b - a)/2
    pho = gamma ** 2
    
    #Intialise Q as vector of zeros 
    Q = np.zeros((N * R, N * R))
    
    # First term 
    for i in range(N):
        #print(f'i is {i}')
        for r in range(R):
            #print(f'r is {r}')
            for j in range(N):
                # print(f'j is {j}')
                for s in range(R):
                    # print(f's is {s}')
                    for k in range(N):
                        # print(f'k is {k}')
                        
                        # Update relevant entry of Q
                        Q[i * R + r,j * R + s] += (2 ** (-r-s)) * M[k,i] * M[k,j] * pho
                        
                        
    # Second term
    # Loop over each number 
    for i in range(N):
        # Loop over each qubit encoding said number
        for r in range(R):
            # Loop of k for matrix multiplication in index form
            for k in range(N):
                Q[i * R + r, i * R + r] += (2 ** (-r)) * M[k,i] * M[k,i] * -2 * gamma 
                
    # Third term
    # Loop over each number
    for i in range(N):
        # Loop over j for the matrix multiplication
        for j in range(N):
            # Loop over each qubit encoding the number
            for r in range(R):
                Q[i * R + r, i * R + r] +=  (2 ** (-r)) * M[j,i] * y[j]  * -2 * gamma 
                
                
    # Fourth term (necessary?)
    for i in range(N):
        for j in range(N):
            Q
    
    # Make Q upper triangular
    return Q+np.triu(Q)-np.tril(Q)

A = np.array([[2,-1],[3,1]])
B = np.array([3,2])
a_test = -1
b_test = 3
accuracy = 10
Q = H(A,B, R = accuracy, a = a_test, b = b_test)
Q_qubo = qv.utils.matrix_to_qubo(Q)
#Q_qubo.solve_bruteforce()

res = list(qv.sim.anneal_qubo(Q_qubo, num_anneals = 1000).best.state.values())
x1 = res[:accuracy]
x2 = res[accuracy:]

print(ConvertSolution(x1,a_test,b_test))
print(ConvertSolution(x2,a_test,b_test))

0.921875
-1.0


In [15]:
def ConvertSolution(q, a, b):
    '''
    Function to convert an array of qubits into a real number
    '''
    
    # Store length of q vector
    n = np.shape(q)[0]
    
    # Initialise chi
    chi = 0
    
    for i in range(n):
        chi += (2 ** (-i)) * q[i]
                
    gamma = (b - a)/2
    
    x = gamma * chi + a
    
    return x
    
    
    
ConvertSolution([0, 1, 1, 1, 1, 0, 1, 1, 0, 0], -1, 3)

0.921875

In [10]:
A = np.array([[2, -1], [3, 1]])
b = np.array([3,2])

Q = get_Q(2,2, A, b)
print(Q)

qubo = qv.utils.matrix_to_qubo(Q)
print(qubo.pretty_str())

qubo.solve_bruteforce()

res = qv.sim.anneal_qubo(qv.utils.matrix_to_qubo(Q), num_anneals=10)

print(res.best)

[[-11.  52.   0.   0.   2.   4.  -2.  -4.]
 [  0.   4.   0.   0.   4.   8.  -4.  -8.]
 [  0.   0.  37.  52.  -2.  -4.   2.   4.]
 [  0.   0.   0. 100.  -4.  -8.   4.   8.]
 [  0.   0.   0.   0.   4.   8.   0.   0.]
 [  0.   0.   0.   0.   0.  12.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   8.]
 [  0.   0.   0.   0.   0.   0.   0.   4.]]
-11.0 x(0) + 52.0 x(0) x(1) + 2.0 x(0) x(4) + 4.0 x(0) x(5) - 2.0 x(0) x(6) - 4.0 x(0) x(7) + 4.0 x(1) + 4.0 x(1) x(4) + 8.0 x(1) x(5) - 4.0 x(1) x(6) - 8.0 x(1) x(7) + 37.0 x(2) + 52.0 x(2) x(3) - 2.0 x(2) x(4) - 4.0 x(2) x(5) + 2.0 x(2) x(6) + 4.0 x(2) x(7) + 100.0 x(3) - 4.0 x(3) x(4) - 8.0 x(3) x(5) + 4.0 x(3) x(6) + 8.0 x(3) x(7) + 4.0 x(4) + 8.0 x(4) x(5) + 12.0 x(5) + 8.0 x(6) x(7) + 4.0 x(7)
  state: {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0}
  value: -13
   spin: False


In [9]:
A = np.array([[-7, -2], [6, 3]])
b = np.array([-4,-5])

Q = get_Q(3,2, A, b)
print(Q)
qubo = qv.utils.matrix_to_qubo(Q)
print(qubo.pretty_str())

print(qubo.solve_bruteforce())

res = qv.sim.anneal_qubo(qv.utils.matrix_to_qubo(Q), num_anneals=10)

print(res.best)

[[  89.  340.  680.    0.   64.  128.  192. -128. -256.    0.    0.    0.]
 [   0.  348. 1360.    0.  128.  256.  384. -256. -512.    0.    0.    0.]
 [   0.    0. 1457.  340.  872.  384.  576. -384. -768.    0.    0.    0.]
 [   0.    0.    0.  332. 1232. -256. -384.  256.  512.    0.    0.    0.]
 [   0.    0.    0.    0. 1115. -460. -664.  512. 1024.    0.    0.    0.]
 [   0.    0.    0.    0.    0.   80.  208.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.  263.   52.  104.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.   24.  208.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.  152.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
89.0 x(0) + 340.0 x(0) x(1) + 680.0 x(0) x(2) + 64.0 x(0) x(4) + 128.0 x(0) x(5) + 192.0 x(0) x(6) 